In [1]:
%reload_ext sql
%sql sqlite:///test_capital_project.db

'Connected: @test_capital_project.db'

In [ ]:
%%sql
CREATE TABLE balance_sheets (
    unit TEXT,
    assets REAL,
    liabilities REAL
);